# SWE-Bench with AGS (Pure Python)

本 Notebook 演示使用 R2E-Gym **Python API** 配合 AGS 后端完成 SWE-Bench 任务。

1. 加载数据集
2. 创建 AGS 沙箱环境
3. 运行 Agent
4. 计算 Reward
5. 保存 Trajectory

前置条件：
准备LLM api


安装依赖

In [ ]:
!git clone https://github.com/lirong-lirong/R2E-Gym.git R2E-Gym
!git clone https://github.com/lirong-lirong/ags_tool.git ags_tool

%pip install -e './R2E-Gym'
%pip install -e './ags_tool[e2b]'

import ags_tool
print(f"ags_tool OK: {ags_tool.__file__}")

import r2egym.agenthub.runtime.ags as _ags_check
print(f"ags.py loaded from: {_ags_check.__file__}")

Obtaining file:///Users/lirong/Desktop/RL/rllm-ags/cookbook/R2E-Gym
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for r2e-gym (pyproject.toml) ... done
  Created wheel for r2e-gym: filename=r2e_gym-0.1.0-0.editable-py3-none-any.whl size=10399 sha256=9b017676bf2be8e02f4d43e23879b7610f9b47fe879704bf96bfe77d1f1cf1a9
  Stored in directory: /private/var/folders/8y/750tmw5j2cv3fdq3zm2kfw6h0000gn/T/pip-ephem-wheel-cache-p8k85a0s/wheels/a7/72/c7/b0932ee171717b947a20ccbb2b0772b27b46e52a39dbc0cd99
Successfully built r2e-gym
  Attempting uninstall: r2e-gym
    Found existing installation: r2e-gym 0.1.0
    Uninstalling r2e-gym-0.1.0:
      Successfully uninstalled r2e-gym-0.1.0

[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restar

配置环境变量

In [ ]:
import os

config = {
    # LLM
    "LLM_BASE_URL": "http://xxxx:60000/v1/",
    "OPENAI_API_KEY": "sk-xxx",
    "LLM_NAME": "openai/glm-4.7",
    # AGS
    "E2B_API_KEY": "xxxI",
    "TENCENTCLOUD_SECRET_ID": "xxx",
    "TENCENTCLOUD_SECRET_KEY": "xxx",
    "AGS_REGION": "ap-guangzhou",
    "TENCENTCLOUD_REGION": "ap-guangzhou",
    # HuggingFace mirror
    "HF_ENDPOINT": "https://hf-mirror.com",
}

os.environ.update(config)

下载数据集

In [ ]:
#! ags需要提前上传容器镜像，并根据容器镜像创建沙箱工具（模版）
#! 为了简便演示流程，我们提前准备了 R2E-Gym/SWE-Bench-Lite 和 R2E-Gym/R2E-Gym-Subset 数据集的前10个镜像，
#! 并且制作好了沙箱工具，ags_tool 会根据原始镜像找到对应的沙箱工具，并创建沙箱实例
from datasets import load_dataset

ds = load_dataset("R2E-Gym/SWE-Bench-Lite", split="test")
print(f"Dataset size: {len(ds)}")

# ========== 可调参数 (等价于 CLI 的 --start_idx / --k) ==========
START_IDX = 0
K = 1  # 改为 10 即运行前 10 条， 这里设为1作为演示。 
# ================================================================

ds_selected = ds.select(range(START_IDX, START_IDX + K))
print(f"Selected {len(ds_selected)} entries (index {START_IDX} ~ {START_IDX + K - 1})")
for i, entry in enumerate(ds_selected):
    print(f"  [{START_IDX + i}] {entry['repo']}  {entry.get('instance_id', '')}")

Dataset size: 300
Selected 1 entries (index 0 ~ 0)
  [0] astropy/astropy  astropy__astropy-12907


执行SWE-Bench测试

In [ ]:
import time
import os
from pathlib import Path

from r2egym.agenthub.environment.env import EnvArgs, RepoEnv
from r2egym.agenthub.agent.agent import AgentArgs, Agent
from r2egym.logging import setup_logging, INFO

# Output file
output_dir = Path("traj")
output_dir.mkdir(parents=True, exist_ok=True)
output_file = output_dir / "swe-bench-ags-python.jsonl"

# R2E-Gym root (YAML command_files paths are relative to this)
R2E_ROOT = Path("R2E-Gym").resolve()

results = []

for idx, ds_entry in enumerate(ds_selected):
    global_idx = START_IDX + idx
    instance_id = ds_entry.get("instance_id", ds_entry["docker_image"])
    print(f"\n{'='*60}")
    print(f"[{idx+1}/{len(ds_selected)}] index={global_idx}  {instance_id}")
    print(f"{'='*60}")

    logger = setup_logging(
        name=f"swe-{global_idx}",
        level=INFO,
        console=False,  # 设置为True可以在控制台中看到大量轨迹
        log_file=str(output_dir / "swe-bench-ags-python.log"),
    )

    try:
        # 1. Create environment
        env_args = EnvArgs(ds=ds_entry)
        env = RepoEnv(env_args, logger=logger, backend="ags")

        # 2. Load agent config
        agent_args = AgentArgs.from_yaml(
            Path("R2E-Gym/src/r2egym/agenthub/config/r2egym/edit_non_fn_calling.yaml")
        )
        agent_args.llm_name = os.getenv("LLM_NAME", "openai/glm-4.7")

        # Fix relative paths: YAML assumes cwd is R2E-Gym root
        agent_args.command_files = [
            str((R2E_ROOT / p).resolve()) for p in agent_args.command_files
        ]

        # 3. Create & run agent
        agent = Agent(name="EditAgent", args=agent_args, logger=logger)
        trajectory = agent.run(
            env,
            max_steps=60,
            temperature=1.0,
            max_steps_absolute=100,
            use_fn_calling=False,
            scaffold="r2egym",
            max_token_limit=65536,
        )

        # 4. Calculate reward
        reward_start = time.time()
        reward, test_output = env.runtime._calculate_reward(get_test_output=True, timeout=300)
        reward_time = time.time() - reward_start

        # 5. Update trajectory
        trajectory.reward = reward
        trajectory.test_output = test_output
        trajectory.ds = ds_entry
        trajectory.reward_calc_time = reward_time

        # 6. Save (append)
        with open(output_file, "a") as f:
            f.write(trajectory.model_dump_json() + "\n")

        results.append({"idx": global_idx, "instance": instance_id, "reward": reward, "steps": trajectory.num_steps})
        print(f"  Reward: {reward}  Steps: {trajectory.num_steps}  Exit: {trajectory.exit_reason}")

    except Exception as e:
        logger.error(f"Failed on {instance_id}: {e}")
        results.append({"idx": global_idx, "instance": instance_id, "reward": None, "steps": None})
    finally:
        try:
            env.close()
        except Exception:
            pass


log_path = output_dir / "swe-bench-ags-python.log"                                                             
if log_path.exists():                                                                                          
    print(log_path.read_text())                                                                                
else:                                                                       
    print("No log file found.")
# Summary
print(f"\n{'='*60}")
print(f"Results saved to {output_file}")
print(f"Detailed logs: {output_dir / 'swe-bench-ags-python.log'}")
print(f"{'='*60}")
for r in results:
    status = f"reward={r['reward']}, steps={r['steps']}" if r["reward"] is not None else "FAILED"
    print(f"  [{r['idx']}] {r['instance']}: {status}")



[1/1] index=0  astropy__astropy-12907
✅ AGS 客户端创建成功
🚀 Creating e2b sandbox with template: astropy__astropy-12907
⏱️  Timeout: 3600s
🌐 Domain: ap-guangzhou.tencentags.com
✅ Sandbox created: 8113ef9a5b18f58e02830a886d550c45ceae0c3f
✅ AGS 客户端创建成功
🚀 Creating e2b sandbox with template: astropy__astropy-12907
⏱️  Timeout: 3600s
🌐 Domain: ap-guangzhou.tencentags.com
✅ Sandbox created: 3a5d22d8daf18298725e0aa32d223a8762e46f45


23:06:19 - LiteLLM:INFO: utils.py:3879 - 
LiteLLM completion() model= glm-4.7; provider = openai
23:06:23 - LiteLLM:INFO: utils.py:1629 - Wrapper: Completed Call, calling success_handler
23:06:23 - LiteLLM:INFO: utils.py:3879 - 
LiteLLM completion() model= glm-4.7; provider = openai
23:06:24 - LiteLLM:INFO: utils.py:1629 - Wrapper: Completed Call, calling success_handler
23:06:24 - LiteLLM:INFO: utils.py:3879 - 
LiteLLM completion() model= glm-4.7; provider = openai
23:06:25 - LiteLLM:INFO: utils.py:1629 - Wrapper: Completed Call, calling success_handler
23:06:25 - LiteLLM:INFO: utils.py:3879 - 
LiteLLM completion() model= glm-4.7; provider = openai
23:06:26 - LiteLLM:INFO: utils.py:1629 - Wrapper: Completed Call, calling success_handler
23:06:26 - LiteLLM:INFO: utils.py:3879 - 
LiteLLM completion() model= glm-4.7; provider = openai
23:06:27 - LiteLLM:INFO: utils.py:1629 - Wrapper: Completed Call, calling success_handler
23:06:28 - LiteLLM:INFO: utils.py:3879 - 
LiteLLM completion() mo

  Reward: False  Steps: 59  Exit: agent
2026-02-10 23:03:10 [INFO] ags.py:223  AGS sandbox started: 8b604818f3767de0166b0aefb531f6881e4ba30f
2026-02-10 23:03:13 [INFO] ags.py:158  AGS environment initialized
2026-02-10 23:03:13 [INFO] ags.py:159  repo name: astropy/astropy
2026-02-10 23:03:13 [INFO] ags.py:160  Docker image: slimshetty/swebench-lite:sweb.eval.x86_64.astropy__astropy-12907
2026-02-10 23:03:13 [INFO] ags.py:161  Tool name: astropy__astropy-12907
2026-02-10 23:03:13 [INFO] ags.py:163  Sandbox ID: 8b604818f3767de0166b0aefb531f6881e4ba30f
2026-02-10 23:03:13 [INFO] env.py:83  Initialized Env: astropy/astropy with image: slimshetty/swebench-lite:sweb.eval.x86_64.astropy__astropy-12907
2026-02-10 23:03:13 [INFO] agent.py:81  Initialized Agent: EditAgent with LLM: openai/glm-4.7
2026-02-10 23:03:13 [WARNING] agent.py:338  Using fn calling: False
2026-02-10 23:03:13 [INFO] agent.py:341  Running agent EditAgent in environment <r2egym.agenthub.environment.env.RepoEnv object at 0x